In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [2]:
(ds_train, ds_test), ds_info = tfds. load (
    "mnist", #it is name that is in catelog of link above
    split=["train", "test"],
    shuffle_files=True, #tf datasets store something in records in multiple files, for easy stream, loading simultaneously while training
                        # so it is for shuffles dataponits in batch so not sequential
    as_supervised=True, #it return tuple (image,label) otherwise it returns dict
    with_info=True, #for getting info

)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
def normalize_image(image, label):
  return tf.cast(image, tf.float32)/255.0, label

AUTOTUNE=tf.data.experimental.AUTOTUNE #tf allow us to give this to set automatically
BATCH_SIZE=32

ds_train=ds_train.map(normalize_image, num_parallel_calls=AUTOTUNE)
ds_train= ds_train.cache() #it keep next data in memory so it will be fast
ds_train=ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train=ds_train.batch(BATCH_SIZE)
ds_train=ds_train.prefetch(AUTOTUNE)

In [4]:
ds_test=ds_test.map(normalize_image, num_parallel_calls=AUTOTUNE)
ds_test=ds_test.batch(128)
ds_test=ds_test.prefetch(AUTOTUNE)

In [5]:
model = keras. Sequential([
    keras. Input ((28, 28, 1)),
    layers.Conv2D (16, 3, padding="same", activation="relu"),
    layers.Flatten(),
    layers. Dense (64, activation="relu"),
    layers. Dense (10),
])

In [6]:
epochs = 5
optimizer = keras.optimizers .Adam ()
loss_fn = keras. losses. SparseCategoricalCrossentropy (from_logits=True)
acc_metric = keras.metrics. SparseCategoricalAccuracy ()


In [ ]:
# Training Loop
for epoch in range (epochs) :
  print (f"\nStart of Training Epoch {epoch}")
  for batch_idx, (x_batch, y_batch) in enumerate (ds_train) :
    with tf.GradientTape() as tape:
      y_pred = model(x_batch, training=True)
      loss = loss_fn (y_batch, y_pred)

    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model. trainable_weights))
    acc_metric.update_state (y_batch, y_pred)

  train_acc = acc_metric.result()
  print (f"Accuracy over epoch {train_acc}")
  acc_metric.reset_states()


Start of Training Epoch 0
Accuracy over epoch 0.9494333267211914

Start of Training Epoch 1
Accuracy over epoch 0.9814833402633667

Start of Training Epoch 2
Accuracy over epoch 0.9879500269889832

Start of Training Epoch 3


In [ ]:
# Test Loop
for batch_idx, (x_batch, y_batch) in enumerate (ds_test) :
  y_pred = model(x_batch, training=True)
  acc_metric.update_state (y_batch, y_pred)

train_acc = acc_metric.result ()
print (f"Accuracy over Test Set: {train_acc}")
acc_metric.reset_states()